In [10]:
import requests
import json
import pandas as pd

# URL ENDPOINT
url = 'https://apps.distelsa.net/FlujoSku/Proveedores/cargarEMCs'

# DEFINICION DE HEADERS CON COOKIE ARRAFINITY Y ASP.NET_SESSIONID
headers = {
    'Cookie': 'ARRAffinity=c4ba1e2211e8bb24de1d78e35c68d1196e2f4c3e25e3040ec9f1c47f0412a550; ASP.NET_SessionId=gwxxpdbylqgdvn2fx4fo2zia',
    'Accept': 'application/json',
}

def fetch_and_parse_data(id_list):
    all_results = []
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    for id_value in id_list:
        params = {'id': str(id_value)}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            try:
                data = response.json()
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON for ID {id_value}: {e}")
                print(f"Response content: {response.text}")
                continue
            
            if data.get('EMCs') is None or data.get('EMC_values') is None:
                print(f"Error en la solicitud para ID {id_value}: Datos EMCs o EMC_values son null")
                continue
            print(f"La solicitud para ID {id_value}: Datos EMCs o EMC_values NO son null")
            
            try:
                emc_values = json.loads(data['EMC_values'])
                emcs = json.loads(data['EMCs'])[0]['attributes']
            except json.JSONDecodeError as e:
                print(f"Error decoding EMCs or EMC_values for ID {id_value}: {e}")
                print(f"EMC_values content: {data.get('EMC_values')}")
                print(f"EMCs content: {data.get('EMCs')}")
                continue
            
            resultado_labels = []

            for emc in emcs:
                attribute_code = emc['attribute_code']
                frontend_label = emc['frontend_label']
                valor_guardado = emc_values.get(attribute_code, "/")

                if emc['frontend_input'] in ["select", "boolean"]:
                    valor_guardado_label = ""
                    for option in emc['options']:
                        if str(option['value']) == str(valor_guardado):
                            valor_guardado_label = option['label']
                            break
                    resultado_labels.append((id_value, frontend_label, valor_guardado_label))
                else:
                    resultado_labels.append((id_value, frontend_label, valor_guardado))

            all_results.extend(resultado_labels)

    df = pd.DataFrame(all_results, columns=['ID', 'Label', 'Valor'])
    return df

# Lista de IDs
id_list = [1110, 4191, 4820, 4588, 4500, 2825, 19933, 25460, 2888, 4820, 4588, 4500, 2825]

# Ejecutar la función para la lista de IDs
df = fetch_and_parse_data(id_list)
print(df)

# Guardar el DataFrame en un archivo CSV
save_path = 'C:/Users/amorejon/OneDrive - Distelsa/Escritorio/PRODUCTOS SIMILARES/PRODUCTOS-SIMILARES-/emc_data.csv'
df.to_csv(save_path, index=False)

La solicitud para ID 1110: Datos EMCs o EMC_values NO son null
Error en la solicitud para ID 4191: Datos EMCs o EMC_values son null
La solicitud para ID 4820: Datos EMCs o EMC_values NO son null
La solicitud para ID 4588: Datos EMCs o EMC_values NO son null
Error en la solicitud para ID 4500: Datos EMCs o EMC_values son null
La solicitud para ID 2825: Datos EMCs o EMC_values NO son null
La solicitud para ID 19933: Datos EMCs o EMC_values NO son null
La solicitud para ID 25460: Datos EMCs o EMC_values NO son null
La solicitud para ID 2888: Datos EMCs o EMC_values NO son null
Error decoding EMCs or EMC_values for ID 2888: Expecting ',' delimiter: line 1 column 45 (char 44)
EMC_values content: {"tipo_material":"4308","tamanio":"Hasta 15"","color":"2575"}
EMCs content: [{"attribute_group_id":"10439","attribute_group_name":"Especificaciones","attribute_group_code":"especificaciones","sort_order":"109","default_id":"0","tab_group_code":"basic","attributes":[{"attribute_id":"135","entity_type